In [ ]:
from src.Normalizer import normalize_file
from src.Tokenizer import Tokenizer
from src.NgramModel import NGModel
import numpy as np

In [2]:
af_files = 'data/train.af.txt'
en_files = 'data/train.en.txt'
nl_files = 'data/train.nl.txt'
xh_files = 'data/train.xh.txt'
zu_files = 'data/train.zu.txt'

norm_af_files = 'data/normalized.af.txt'
norm_en_files = 'data/normalized.en.txt'
norm_nl_files = 'data/normalized.nl.txt'
norm_xh_files = 'data/normalized.xh.txt'
norm_zu_files = 'data/normalized.zu.txt'

# normalize_file(af_files, norm_af_files)
# normalize_file(en_files, norm_en_files)
# normalize_file(nl_files, norm_nl_files)
# normalize_file(xh_files, norm_xh_files)
# normalize_file(zu_files, norm_zu_files)

Document Normalized successfully!
Document Normalized successfully!
Document Normalized successfully!
Document Normalized successfully!
Document Normalized successfully!


In [3]:
af_trigram = NGModel(norm_af_files, 3)
en_trigram = NGModel(norm_en_files, 3)
nl_trigram = NGModel(norm_nl_files, 3)
xh_trigram = NGModel(norm_xh_files, 3)
zu_trigram = NGModel(norm_zu_files, 3)

In [7]:
val_af_files = 'data/val.af.txt'
val_en_files = 'data/val.en.txt'
val_nl_files = 'data/val.nl.txt'
val_xh_files = 'data/val.xh.txt'
val_zu_files = 'data/val.zu.txt'

val_af_token = Tokenizer(val_af_files)
val_en_token = Tokenizer(val_en_files)
val_nl_token = Tokenizer(val_nl_files)
val_xh_token = Tokenizer(val_xh_files)
val_zu_token = Tokenizer(val_zu_files)

val_af_trigram = val_af_token.build_ngram(3)
val_en_trigram = val_en_token.build_ngram(3)
val_nl_trigram = val_nl_token.build_ngram(3)
val_xh_trigram = val_xh_token.build_ngram(3)
val_zu_trigram = val_zu_token.build_ngram(3)

In [8]:
val_af_trigram

{'\n',
 ' ',
 '/',
 '0',
 '<',
 '>',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
def get_score(text):
    score = {
        'af': 0,
        'en': 0,
        'nl': 0,
        'xh': 0,
        'zu': 0
    }
    for trigram in text.keys():
        score['af'] += af_trigram_model.get(trigram, 0)
        score['en'] += en_trigram_model.get(trigram, 0)
        score['nl'] += nl_trigram_model.get(trigram, 0)
        score['xh'] += xh_trigram_model.get(trigram, 0)
        score['zu'] += zu_trigram_model.get(trigram, 0)
    # Softmax
    score = {k:np.exp(v) for k,v in score.items()}
    w = sum(x for x in score.values())
    score = {k:np.round(v/w, 5) for k,v in score.items()}
    return sort_dict(score)

In [ ]:
print(get_score(val_af_trigram))
print(get_score(val_en_trigram))
print(get_score(val_nl_trigram))
print(get_score(val_xh_trigram))
print(get_score(val_zu_trigram))

In [ ]:
th = {k:np.round(v,7) for k, v in en_trigram_model.items() if k.startswith('th')}
sort_dict(th)

In [ ]:
test =  "rakoto"
f